# 四、决策树相关算法模型

## 1.  引用包

In [20]:
#-*- coding: utf-8 -*-
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier, plot_tree
import pydotplus
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from itertools import product
from IPython.display import Image
import re

## 2. 数据处理 

In [21]:
### （1）读数据和简单筛选 ###

# 1. 从⽂文件中读取原始数据
file_spread = '../数据集/债明细-生数据集.xlsx'           ###
df=pd.read_excel(file_spread)

# 2. 按照指标分别删除0行
titles = ['ROE', 'ROA', 'ROS', 'ROIC', '总资产周转率','流动资产周转率','存货周转率','应收账款周转率','资产负债率','流动比率','速动比率','现金比率','利息保障倍数','销售收入增长率','净利润增长率','总资产增长率','经营活动现金流净额占比','投资活动现金流净额占比','筹资活动现金流净额占比','国内生产总值','居民消费价格指数增长率','一般预算收入','spread','scoring']
titles2 = ['ROE', 'ROA', 'ROS', 'ROIC', '总资产周转率','流动资产周转率','存货周转率','应收账款周转率','资产负债率','流动比率','速动比率','现金比率','利息保障倍数','销售收入增长率','净利润增长率','总资产增长率','经营活动现金流净额占比','投资活动现金流净额占比','筹资活动现金流净额占比','国内生产总值','居民消费价格指数增长率','一般预算收入','spread']
for t in titles:
    if t != '居民消费价格指数增长率':
        df = df[df[t]!=0]

# 3. 删除不必要
df.drop(['发行人','发行规模','债券余额','发行期限','票面利率(发行参考)','上市日期','起息日','到期日','发行人省份','城市','起息日期'],axis=1,inplace=True)

# 4. df乱序
df = df.reindex(np.random.permutation(df.index))

In [22]:
# 5. 存取筛选后的数据
ROE = df[titles[0]].values.tolist()   #盈利能力
ROA = df[titles[1]].values.tolist()
ROS = df[titles[2]].values.tolist() 
ROIC = df[titles[3]].values.tolist()

total_turnover = df[titles[4]].values.tolist()  #营运能力
flow_turnover = df[titles[5]].values.tolist()
inventory_turnover = df[titles[6]].values.tolist()
receive_turnover = df[titles[7]].values.tolist()

debt_r = df[titles[8]].values.tolist()   #负债情况
flow_r = df[titles[9]].values.tolist()
quick_r = df[titles[10]].values.tolist()
cash_r = df[titles[11]].values.tolist()
interest_mul = df[titles[12]].values.tolist()

sales = df[titles[13]].values.tolist()  #成长能力
profit = df[titles[14]].values.tolist()
assets = df[titles[15]].values.tolist()

business = df[titles[16]].values.tolist()   #现金流情况
invest = df[titles[17]].values.tolist()
finance = df[titles[18]].values.tolist() 

gdp = df[titles[19]].values.tolist()   #宏观
cpi = df[titles[20]].values.tolist()
budget = df[titles[21]].values.tolist()
spread = df[titles[22]].values.tolist()

# 6. 列重命名
df.rename(columns={'总资产周转率':'TAT','流动资产周转率':'CAT','存货周转率':'ITR','应收账款周转率':'RTR', 
                   '资产负债率':'Debt','流动比率':'Flow','速动比率':'Quick','现金比率':'Cash','利息保障倍数':'Interest',
                   '销售收入增长率':'Sales','净利润增长率':'Profit','总资产增长率':'Assets',
                   '经营活动现金流净额占比':'Business','投资活动现金流净额占比':'Invest','筹资活动现金流净额占比':'Finance',
                   '国内生产总值':'GDP','居民消费价格指数增长率':'CPI','一般预算收入':'Budget'
                  }, inplace = True)

# 7. 转为array形式
dataset = np.array(df)

In [23]:
# 8. 转换数据集到iris.data格式
dataset = []
list_len = len(df[titles[0]].values.tolist())
for j in range(list_len):
    tmp_list = []
    tmp_list.append(ROE[j])
    tmp_list.append(ROA[j])
    tmp_list.append(ROS[j])
    tmp_list.append(ROIC[j])
    tmp_list.append(total_turnover[j])
    tmp_list.append(flow_turnover[j])
    tmp_list.append(inventory_turnover[j])
    tmp_list.append(receive_turnover[j])
    tmp_list.append(debt_r[j])
    tmp_list.append(flow_r[j])
    tmp_list.append(quick_r[j])
    tmp_list.append(cash_r[j])
    tmp_list.append(interest_mul[j])
    tmp_list.append(sales[j])
    tmp_list.append(profit[j])
    tmp_list.append(assets[j])
    tmp_list.append(business[j])
    tmp_list.append(invest[j])
    tmp_list.append(finance[j])
    tmp_list.append(gdp[j])
    tmp_list.append(cpi[j])
    tmp_list.append(budget[j])
    tmp_list.append(spread[j])
    
    dataset.append(tmp_list)
        
data_array = np.array(dataset)
print(data_array)                                                                                             

[[1.51190000e+00 1.42520000e+00 1.63373000e+01 ... 6.82926829e-03
  6.48539590e+07 2.14000000e+00]
 [1.57580000e+00 1.49900000e+00 1.20159000e+01 ... 6.86274510e-03
  3.04867050e+07 9.50000000e-01]
 [3.42060000e+00 1.87470000e+00 1.82252000e+01 ... 6.82926829e-03
  6.48539590e+07 1.72700000e+00]
 ...
 [2.41890000e+00 3.26100000e+00 1.19073000e+01 ... 1.08374384e-02
  3.00740870e+07 1.09900000e+00]
 [1.17480000e+00 2.29350000e+00 2.78437000e+01 ... 1.17762512e-02
  3.30707820e+07 1.26610000e+00]
 [6.71100000e-01 8.92200000e-01 2.49706000e+01 ... 6.86274510e-03
  2.10623970e+07 7.28200000e-01]]


In [24]:
# 9. 转换数据评级到iris.target格式
scoring = df[titles[23]].values.tolist()

## 3. 构建决策树和验证

In [25]:
# 1. 分别将data和target转回dataFrame格式
x = pd.DataFrame(data_array)  
y = pd.DataFrame(scoring)

# 2. 分割数据集，一部分测试一部分训练，数据集的分割，分为训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
dict = DictVectorizer(sparse=False)                             # 进行特征工程的处理，将数据集变换成one-hot形式
x_train = dict.fit_transform(x_train.to_dict(orient="records")) # 将列表转换为一个个字典形式，因为DictVectorizer(）中接收的是字典形式
print("feature name: ",dict.get_feature_names())
x_test = dict.transform(x_test.to_dict(orient="records"))
tre=DecisionTreeClassifier()  # 用决策数预测
tre.fit(x_train,y_train)

# 3. 输出准确度
print(tre.score(x_test, y_test))

# 4. 评估模型使用十次交叉验证
score = cross_val_score(tre, x, y, cv=10, scoring='accuracy')
print(np.mean(score))

feature name:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
0.6098378982671884


/Library/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


0.6517565878320426


## 4. 决策树可视化 

In [26]:
results = []
for i in range(22):
    results.append(str(i))

# 1. graphviz插件实现决策树可视化
dot_data = tree.export_graphviz(tre, out_file=None,
                         feature_names=titles2,
                         class_names=results,
                         filled=True, rounded=True,
                         special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data)

# 2. 写入决策树图像
graph.write_pdf("../数据集/results/Decision Tree Visualization.pdf") 

True

## 5. 实现预测

In [27]:
# 1. 参数输入
pre_set = ([[1.504, 2.5236, 29.0848, 2.9762, 0.0708, 0.1869, 1.1383, 0.5664, 2.715, 1.75, 1.55, 0.59, 1.28, 14.93, 7.57, 19.10, 14.780, -188.14, 273.364, 1714.66, 0.0068, 40386705, 0.54]])

# 2. 输出原格式为 [18]， 需要进行处理后提取
pre_result = tre.predict(pre_set)
score = re.sub("\D","", str(pre_result))

# 3. scoring和评级转换
scoring_dict = {'21':'AAA', '20':'AAA-', 
        '19':'AA+','18':'AA','17':'AA-',
        '16':'A+', '15':'A', '14':'A-', 
        '13':'BBB+', '12':'BBB', '11':'BBB-',
        '10':'BB+', '9':'BB', '8':'BB-',
        '7':'B+', '6':'B', '5':'B-', 
        '4':'CCC+', '3':'CCC', '2':'CCC-', '1':'CC', '0':'C'}  
print("预测结果scoring：", scoring_dict[score])

预测结果scoring： AA
